In [1]:
import pandas as pd
import numpy as np
from tqdm import trange

In [2]:
# Read data
train = pd.read_csv("train.data.csv")
test = pd.read_csv("test.data.csv")

In [3]:
# Extract useful variables
X_train = train[['bedrooms','bathrooms', 'sqft_living', 'sqft_lot']].to_numpy()
X_test = test[['bedrooms','bathrooms', 'sqft_living', 'sqft_lot']].to_numpy()
Y_train = train['price'].to_numpy().reshape(-1,1)
Y_test = test['price'].to_numpy().reshape(-1,1)

In [4]:
# Add interaction
X_train = np.append(X_train, (X_train[:,0]*X_train[:,1]).reshape(-1,1),axis=1)
X_test = np.append(X_test, (X_test[:,0]*X_test[:,1]).reshape(-1,1),axis=1)
# Data standardization.
# Since our data is standardized, there is no need to include intercept term.
X_train_mean = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)
Y_train_mean = np.mean(Y_train)
Y_train_std = np.std(Y_train)

X1_train = (X_train-X_train_mean)/X_train_std
X1_test = (X_test-X_train_mean)/X_train_std
Y1_train = (Y_train-Y_train_mean)/Y_train_std
Y1_test = (Y_test-Y_train_mean)/Y_train_std

(a)\
We use the formulae $\hat{\beta}=(X^T X)^{-1}X^T Y$, $R_2=1-\frac{SSE}{SSTO}$

In [5]:
def OLS(X, Y, X1, Y1):
    beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
    SSE_train = np.sum((Y-X.dot(beta))**2)
    SSTO_train = np.sum((Y-np.mean(Y))**2)
    R2_train = 1 - SSE_train/SSTO_train
    SSE_test = np.sum((Y1-X1.dot(beta))**2)
    SSTO_test = np.sum((Y1-np.mean(Y1))**2)
    R2_test = 1 - SSE_test/SSTO_test
    return beta, R2_train, R2_test

In [6]:
beta1, R2_train1, R2_test1 = OLS(X1_train[:,:-1], Y1_train, X1_test[:,:-1], Y1_test)
intercept = Y_train_mean-np.sum(np.squeeze(beta1) * X_train_mean[:-1]/X_train_std[:-1])*Y_train_std
coef = np.squeeze(beta1)/X_train_std[:-1]*Y_train_std
print("Our model is Y={0}{1}*X1+{2}*X2+{3}*X3{4}*X4.".format(np.round(intercept,5),np.round(coef[0],5),np.round(coef[1],5),np.round(coef[2],5),np.round(coef[3],5)))
print("R2 on train set is {0}.\nR2 on test set is {1}.".format(np.round(R2_train1,5),np.round(R2_test1,5)))

Our model is Y=80832.98912-59296.95658*X1+3681.6562*X2+316.68573*X3-0.42674*X4.
R2 on train set is 0.51011.
R2 on test set is 0.50499.


(b)

In [7]:
fancy = pd.read_csv("fancyhouse.csv")
fancy_matrix = (fancy[['bedrooms','bathrooms', 'sqft_living', 'sqft_lot']].to_numpy()-X_train_mean[:-1])/X_train_std[:-1]
fancy_predict = fancy_matrix.dot(beta1)*Y_train_std + Y_train_mean
print("Predicted price for Bill Gats's house is {}$".format(np.round(fancy_predict[0,0],3)))

Predicted price for Bill Gats's house is 15436769.538$


We think this price is okay.

(c)

In [8]:
beta2, R2_train2, R2_test2 = OLS(X1_train, Y1_train, X1_test, Y1_test)
intercept1 = Y_train_mean-np.sum(np.squeeze(beta2)*X_train_mean/X_train_std)*Y_train_std
coef1 = np.squeeze(beta2)/X_train_std*Y_train_std
print("Our model is Y={0}{1}*X1{2}*X2+{3}*X3{4}*X4+{5}*X1*X2.".format(np.round(intercept1,5),np.round(coef1[0],5),np.round(coef1[1],5),np.round(coef1[2],5),np.round(coef1[3],5),np.round(coef1[4],5)))
print("R2 on train set is {0}.\nR2 on test set is {1}.".format(np.round(R2_train2,5),np.round(R2_test2,5)))

Our model is Y=320752.74798-128706.40872*X1-111008.86939*X2+308.72484*X3-0.42458*X4+33753.60514*X1*X2.
R2 on train set is 0.51735.
R2 on test set is 0.51054.


(d)

In [9]:
def gradient_descend(X, Y, stepsize, epsilon=1e-8):
    _, p = X.shape
    XTX = X.T.dot(X)
    XTY = X.T.dot(Y)
    beta = np.random.randn(p,1)
    i = 0
    while 1:
        g = XTX.dot(beta)-XTY
        if np.linalg.norm(g) < epsilon:
            break
        i = i + 1
        beta = beta - stepsize*g
    print("Using {} iteractions to converge.".format(i))
    return beta

In [10]:
# Choose stepsize according to L and mu.
# Since this problem is strongly convex, L can be the max eigenvalue while mu can be the minimal eigenvalue
# learning rate is 2/(lambda_min+lambda_max)
lamda1 = np.linalg.eig(X1_train[:,:-1].T.dot(X1_train[:,:-1]))[0]
lamda2 = np.linalg.eig(X1_train.T.dot(X1_train))[0]
np.random.seed(2022)
beta3 = gradient_descend(X1_train[:,:-1],Y1_train,stepsize=2/(lamda1[0]+lamda1[-1]))
beta4 = gradient_descend(X1_train,Y1_train,stepsize=2/(lamda2[0]+lamda2[-1]))

Using 135 iteractions to converge.
Using 1409 iteractions to converge.


In [11]:
# We can see they are almost the same.
print("Beta for (a) using matrix operation: {}".format(np.squeeze(beta1)))
print("Beta for (a) using gradient descent: {}".format(np.squeeze(beta3)))
print("Beta for (c) using matrix operation: {}".format(np.squeeze(beta2)))
print("Beta for (c) using gradient descent: {}".format(np.squeeze(beta4)))

Beta for (a) using matrix operation: [-0.15147017  0.00773629  0.7918328  -0.04514359]
Beta for (a) using gradient descent: [-0.15147017  0.00773629  0.7918328  -0.04514359]
Beta for (c) using matrix operation: [-0.32877205 -0.23326364  0.77192759 -0.04491517  0.38964848]
Beta for (c) using gradient descent: [-0.32877205 -0.23326364  0.77192759 -0.04491517  0.38964848]


(e)

In [12]:
def stochastic_gradient_descend(X, Y, stepsize, max_epoch):
    n, p = X.shape
    beta = np.random.randn(p)
    iter = 0
    with trange(max_epoch) as pbar:
        for i in pbar:
            indice = np.random.permutation(n)
            for j in range(n):
                idx = indice[j]
                g = -2*X[idx]*(Y[idx]-np.dot(beta,X[idx]))
                beta = beta - g*stepsize/(iter+1)
                iter += 1
            if i % 1 == 0:
                MSE = np.mean((Y[:,0]-X.dot(beta))**2)
                pbar.set_description("MSE={}".format(np.round(MSE,5)))
    return beta

In [13]:
np.random.seed(2022)
n = X_train.shape[0]
beta5 = stochastic_gradient_descend(X1_train[:,:-1],Y1_train,stepsize=1e-2,max_epoch=200)

MSE=3.15989: 100%|███████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.42it/s]


In [17]:
beta6 = stochastic_gradient_descend(X1_train,Y1_train,stepsize=1e-5,max_epoch=500)

MSE=0.48265: 100%|██████████| 500/500 [02:15<00:00,  3.70it/s]


In [18]:
print("Beta for (a) using matrix operation: \n{}".format(np.squeeze(beta1)))
print("Beta for (a) using sgd: \n{}".format(np.squeeze(beta5)))
print("Beta for (c) using matrix operation: \n{}".format(np.squeeze(beta2)))
print("Beta for (c) using sgd: \n{}".format(np.squeeze(beta6)))

Beta for (a) using matrix operation: 
[-0.15147017  0.00773629  0.7918328  -0.04514359]
Beta for (a) using sgd: 
[-0.15070764  0.00874859  0.7927156  -0.04433098]
Beta for (c) using matrix operation: 
[-0.32877205 -0.23326364  0.77192759 -0.04491517  0.38964848]
Beta for (c) using sgd: 
[-0.32410012 -0.22732823  0.77209111 -0.04496272  0.3810138 ]


In [19]:
def R2(X, Y, beta):
    SSE = np.sum((Y-X.dot(beta.reshape(-1,1)))**2)
    SSTO = np.sum((Y-np.mean(Y))**2)
    return 1 - SSE/SSTO

In [20]:
R2_train_sgd = R2(X1_train[:,:-1],Y1_train,beta5)
R2_test_sgd = R2(X1_test[:,:-1],Y1_test,beta5)
R2_train_int_sgd = R2(X1_train,Y1_train,beta6)
R2_test_int_sgd = R2(X1_test,Y1_test,beta6)
print("R2 for SGD is:\nModel without interaction train {0}, test {1}\nModel with interaction train {2}, test {3}".format(np.round(R2_train_sgd,5),np.round(R2_test_sgd,5),np.round(R2_train_int_sgd,5),np.round(R2_test_int_sgd,5)))

R2 for SGD is:
Model without interaction train 0.51011, test 0.50501
Model with interaction train 0.51735, test 0.51055
